# Adversarial Simulator

###### This notebook demonstrates a simulated adversarial data creation and evaluation.

In [ ]:
# Install the packages
# %pip install azure-ai-generative[simulator,evaluate] --upgrade

In [ ]:
#Get Azure Key Vault Client
key_vault_name = 'kv_to-be-replaced'

In [ ]:
from azure.keyvault.secrets import SecretClient  
from azure.identity import DefaultAzureCredential  

def get_secrets_from_kv(kv_name, secret_name):
    
  # Set the name of the Azure Key Vault  
  key_vault_name = kv_name 
    
  # Create a credential object using the default Azure credentials  
  credential = DefaultAzureCredential()

  # Create a secret client object using the credential and Key Vault name  
  secret_client = SecretClient(vault_url=f"https://{key_vault_name}.vault.azure.net/", credential=credential)  
    
  # Retrieve the secret value  
  return(secret_client.get_secret(secret_name).value)

### Parameters and imports

In [ ]:
from pathlib import Path
from azure.ai.generative.evaluate import evaluate
import json
from azure.ai.generative.synthetic.simulator import Simulator
from azure.ai.resources.client import AIClient
from azure.identity import DefaultAzureCredential
from azure.ai.resources.entities import AzureOpenAIModelConfiguration
from openai import AsyncAzureOpenAI

sub = get_secrets_from_kv(key_vault_name,"AZURE-SUBSCRIPTION-ID") #"19b29b25-a38c-443f-bc1f-e0aaf8e55116"
rg = get_secrets_from_kv(key_vault_name,"AZURE-RESOURCE-GROUP") #"ncbyc-red-teaming-rg"
project_name = 'ai_studio_project_name_to-be-replaced'

oai_key = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-KEY")
oai_endpoint=  get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-ENDPOINT")
oai_api_version = "2023-12-01-preview" #get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-PREVIEW-API-VERSION")

oai_client = AsyncAzureOpenAI(api_key=oai_key, 
                              azure_endpoint=oai_endpoint, 
                              api_version=oai_api_version)

In [ ]:
async_oai_chat_completion_fn = oai_client.chat.completions.create
client = AIClient(
    subscription_id=sub, resource_group_name=rg, project_name=project_name, credential=DefaultAzureCredential()
)

## Simulate the data

In [ ]:
simulator = Simulator.from_fn(fn=async_oai_chat_completion_fn, ai_client=client, model="gpt-4", max_tokens=300)
adv_template = Simulator.get_template("adv_qa")

## Run the simulator

In [ ]:
outputs = await simulator.simulate_async(
    adv_template, max_conversation_turns=1, simulation_result_limit=500, api_call_delay_sec=10
)

## Print the outputs

In [ ]:
# for line in outputs:
#     print(json.dumps(line, indent=2))

## Store outputs in file for eval

In [ ]:
eval_name = "conv-oai-qa-harm-eval_500"
file_name = f"{eval_name}.jsonl"
jsonl_object = outputs.to_eval_qa_json_lines()
with Path.open(file_name, "w") as f:
    f.write(jsonl_object)

## Run evaluate

In [ ]:
import urllib.request
import json
import os
import ssl

def qa_fn(chat_history=[], question='', **kwargs):
    
    data = {"chat_history":[],"query": question}

    body = str.encode(json.dumps(data))

    url = 'ai_studio_project_name_to-be-replaced'#get_secrets_from_kv(key_vault_name,"AI_STUDIO_DRAFT_FLOW_ENDPOINT")  #prompt flow endpoint url
    api_key = 'ai_studio_project_name_to-be-replaced' #get_secrets_from_kv(key_vault_name,"AI_STUDIO_DRAFT_FLOW_API_KEY") #prompt flow endpoint key
    deployment_name = 'ai_studio_project_name_to-be-replaced' #get_secrets_from_kv(key_vault_name,"AI_STUDIO_DRAFT_FLOW_DEPLOYMENT_NAME")

    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': deployment_name } #prompt flow endpoint name

    req = urllib.request.Request(url, body, headers)

    answer = ''
    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        answer = json.loads(result)['reply']
    except urllib.error.HTTPError as error:
        answer = 'The request failed'

    return{
            "question": question,
            "answer": answer
    }

In [ ]:
from azure.ai.generative.evaluate import evaluate

data_path = Path.cwd() / "conv-oai-qa-harm-eval_500.jsonl"
output_path = Path.cwd() / "downloaded_artifacts_500" / "remote"

harms_evaluation = evaluate( 
    evaluation_name="conv-oai-qa-harm-eval_500", 
    data=data_path, # red-teamed dataset including harmful content from your chat app
    task_type="qa",
    metrics_list=["violence", "self_harm", "sexual", "hate_fairness"], #specify content harms metrics
    data_mapping={
            "question": "question",
            #"context": "context",
            "answer": "answer",
            #"ground_truth": "ground_truth"
        },
    output_path=output_path, #optional: save evaluation results .jsonl to local folder path 
    tracking_uri=client.tracking_uri,
    target=qa_fn
)